In [1]:
import numpy as np
import pandas as pd

In [2]:
# load geometry data

from os import listdir, makedirs
from os.path import isfile, join, isdir

def find_files(path_to_folder, condition=None):
	if (condition):
		if (condition[0] == "fileFormat"):
			_l = len(condition[1])
			return [(join(path_to_folder,f), f,f[:-_l - 1]) for f in listdir(path_to_folder) if isfile(join(path_to_folder, f)) and f[-_l:] == condition[1]]
		else:
			raise(ValueError("Condition should be a tuple where the first argument is the condition name and the second is its value"))
	else:
		return [(join(path_to_folder,f), f, f.rsplit(".")[0]) for f in listdir(path_to_folder) if isfile(join(path_to_folder, f))]



# Load Pickles

In [3]:
# import compiled data

compiled_dir = "./compiled_data"
elems_file = "elems_compiled.pickle"
nodes_file = "nodes_compiled.pickle"

e_p = join(compiled_dir, elems_file)
n_p = join(compiled_dir, nodes_file)

elems_df = pd.read_pickle(e_p)
nodes_df = pd.read_pickle(n_p)



# Load geometry data

In [4]:
# get geometry files
import pandas as pd
GEOMETRY_FILES_PATH = "./geometry_data"

files = find_files(GEOMETRY_FILES_PATH)

# sort files
geo_dict = {}
for (fp, ff, fn) in files:
    fs = fn.split("_")
    if fs[-1] == "elems":
        key = fn.replace("_elems","")

        df = pd.read_csv(fp, header=None, sep='\n')
        df = df[0].str.split(",",expand=True)
        columns = ["elem"] + [*range(1,len(df.columns))]
        df.columns = columns
        df = df.apply(pd.to_numeric)

        geo_dict[key] = df.to_dict('records')

In [5]:
geo_dict.keys()

dict_keys(['myo_hex_coarse_8'])

In [6]:
HEADER_KEY = ['fileName']
HEADER_TIM = ['time']
HEADER_STR = ['sx','sy','sz','sxy','sxz','syz']
HEADER_DIS = ['ux','uy','uz']
HEADER_POS = ['x','y','z']
HEADER_FAL = ['fail']

# Calculate centroids
The calcuation of centroid of each element is needed to determine where the element is located. If it is close to the apex, it will have its value reduced
- 1 Get the elemts and each node
- 2 Get x,y,z for each node for each element
- 3 Compute centroid for each element (x1,x2,x3... -> x)
- 4 Add additional data

In [7]:
def get_base_name(fileName, nm=[0,4]):
    splitted_name = fileName.replace("with_load_","").split("_epi")[0].split("_")
    return "_".join([s for s in splitted_name[nm[0]:nm[1]]])

def get_full_name(fileName):
    return "with_load_" + fileName + "_epi_60_endo_-60"

def hash_with_id(basename,id,time):
    return hash(basename + "-" + str(int(id)) + "-" + str(float(time)) )

In [8]:
get_base_name("myo_hex_8_coarse_PAQ")

'myo_hex_8_coarse'

In [9]:
class Modified_dict():
    def __init__(self, dataframe, _type):
        self._type = _type
        self.vec = dataframe.to_dict('records')
        self.dict = self.transform(dataframe)

    def __call__(self):
        return self.dict

    def get(self, base_name, id, time, fileName=None):
        if fileName != None:
            base_name = get_base_name(fileName)
        return self.dict[hash_with_id(base_name, id, time)]

    def transform(self, dataframe, look_for=None):
        new_dict = dict()
        if look_for == None:
            look_for = self._type
        for con in self.vec:
            con_num = con[look_for]
            con_tim = con["time"]
            base_name = get_base_name(con["fileName"])
            hash_name = hash_with_id(base_name, con_num, con_tim)
            new_dict.update({
                hash_name: con
            })
        return new_dict
    



In [10]:
elem_dict = Modified_dict(elems_df, "elem")
node_dict = Modified_dict(nodes_df, "node")


In [11]:
# node_dict.get(_, 74.0, 0.2, fileName="with_load_myo_hex_medium_8_epi_60_endo_-60" )

In [12]:
grouped_elems_df_time = elems_df.groupby(["time"])
e_timesteps = list(grouped_elems_df_time.groups.keys())
print(e_timesteps)

[0.004, 0.2]


In [13]:
grouped_elems_df_time.get_group(0.2)

,time,elem,sx,sy,sz,sxy,sxz,syz,fileName
20000,0.2,1.0,-5.941120,52.3022,-13.7855,-16.729100,1.015420,-3.05725,myo_hex_8_coarse_PAQ
20001,0.2,2.0,-0.126393,47.6513,-13.2595,-23.596200,1.420530,-2.95780,myo_hex_8_coarse_PAQ
20002,0.2,3.0,6.939390,40.5597,-13.4835,-29.207900,1.805290,-2.73696,myo_hex_8_coarse_PAQ
20003,0.2,4.0,15.490000,32.0672,-14.2087,-33.380800,2.190980,-2.48461,myo_hex_8_coarse_PAQ
20004,0.2,5.0,26.938700,24.1022,-13.1274,-35.010500,2.548330,-2.17264,myo_hex_8_coarse_PAQ
...,...,...,...,...,...,...,...,...,...
79995,0.2,19996.0,4.458240,12.3052,27.8219,6.830940,2.449660,4.41341,myo_hex_coarse_8_PAQ
79996,0.2,19997.0,2.611210,13.9552,28.9259,5.530600,2.116080,5.92202,myo_hex_coarse_8_PAQ
79997,0.2,19998.0,1.488700,15.4157,28.9066,4.097190,1.292730,5.50884,myo_hex_coarse_8_PAQ
79998,0.2,19999.0,0.922051,16.6887,28.8449,2.301980,0.525949,5.98947,myo_hex_coarse_8_PAQ


In [14]:
from IPython.display import clear_output, display

grouped_elems_df = elems_df.groupby(["fileName"])
grouped_nodes_df = nodes_df.groupby(["fileName"])
grouped_elems_df_time = elems_df.groupby(["time"])

# grouped_elems_df_ft = elems_df.groupby(["fileName", "time"])

# min_zs = grouped_elems_df_ft.min()["z"]
# max_zs = grouped_elems_df_ft.max()["z"]

e_keys = list(grouped_elems_df.groups.keys())
n_keys = list(grouped_nodes_df.groups.keys())

e_timesteps = list(grouped_elems_df_time.groups.keys())
print()
allowed_keys = e_keys + n_keys


BASE = np.array([0.0, 0.0, 0.0])
EPI_APEX = np.array([0.0, 0.0, -75.0])
ENDO_APEX = np.array([0.0, 0.0, -65.0])

def get_euclidean_dist(a,b):
    return np.linalg.norm(a-b)
def get_dist_to_line(P1, P2, P3):
    return norm(np.cross(P2-P1, P1-P3))/norm(P2-P1)

def get_nodes(geo_data):
    _data = []
    for i in range(1,8):
        if i in geo_data and not np.isnan(geo_data[i]):
            _data.append(geo_data[i])
    return _data

_f_max = len(geo_dict)
for i, base_name in enumerate(geo_dict):
    _e_max = len(geo_dict[base_name])

    if base_name in allowed_keys:
        continue

    for j, geo_data in enumerate(geo_dict[base_name]):
        elem_num = geo_data["elem"]
        elem_nodes = get_nodes(geo_data)
        # print(elem_nodes)
        
        xsum = 0
        ysum = 0 
        zsum = 0
        uxsum = 0
        uysum = 0
        uzsum = 0

        for time_step in e_timesteps:
            for node_num in elem_nodes:
                # print("base_name", base_name, "node_num", node_num)
                try:
                    node = node_dict.get(base_name, node_num, time_step)
                    xsum += node["x"]
                    ysum += node["y"]
                    zsum += node["z"]
                    uxsum += node["ux"]
                    uysum += node["uy"]
                    uzsum += node["uz"]
                except:
                    pass
            
            try:
                elem = elem_dict.get(base_name, elem_num, time_step)

                count = len(elem_nodes)
                xm = xsum / count
                ym = ysum / count
                zm = zsum / count

                cP = np.array([xm, ym, zm])

                elem["x"] = xsum / count
                elem["y"] = ysum / count
                elem["z"] = zsum / count
                elem["ux"] = uxsum / count
                elem["uy"] = uysum / count
                elem["uz"] = uzsum / count
                elem["n_nodes"] = count

                # # calculate distances
                # elem["d_min_z"]  = abs(zm - min_zs[get_full_name(base_name)][time_step])
                # elem["d_max_z"]  = abs(zm - max_zs[get_full_name(base_name)][time_step])
                # elem["d_center"] = ym
                
                # calculate von mises (from https://forums.febio.org/archive/index.php/t-412.html)
                vm = elem["sx"] * elem["sx"] + elem["sy"] * elem["sy"] + elem["sz"] * elem["sz"] 
                vm -= elem["sx"] * elem["sy"] + elem["sy"] * elem["sz"] + elem["sx"] * elem["sz"] 
                vm += 3 * (elem["sxy"] * elem["sxy"] + elem["syz"] * elem["syz"] + elem["sxz"] * elem["sxz"])

                elem["vm"] = np.sqrt(vm)

            except Exception as e: 
                # print(e)
                pass
                
        

        if j % 100 == 0:
            clear_output(wait=True)
            display("Progress: {curr_j} | {e_total} - {curr_i} | {f_total}".format(curr_j=j,curr_i = i, e_total=_e_max, f_total=_f_max))


'Progress: 19900 | 20000 - 0 | 1'

In [15]:
elem_list = list(elem_dict().values())
new_elem_df = pd.DataFrame(elem_list, columns=list(elem_list[0].keys()))

In [16]:
out_dir = "./compiled_data"
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

new_elem_df.to_pickle(join(out_dir, "elems_with_centroid" + ".pickle"))

In [17]:
new_elem_df

,time,elem,sx,sy,sz,sxy,sxz,syz,fileName
0,0.004,1.0,-0.838460,-0.341764,-1.003850,-0.376963,-0.012362,-0.050719,myo_hex_8_coarse_PAQ
1,0.004,2.0,-0.725720,-0.440269,-0.993483,-0.421445,-0.004908,-0.055450,myo_hex_8_coarse_PAQ
2,0.004,3.0,-0.624983,-0.577933,-1.025380,-0.455080,0.001749,-0.054960,myo_hex_8_coarse_PAQ
3,0.004,4.0,-0.484894,-0.707290,-1.042970,-0.463514,0.008015,-0.052948,myo_hex_8_coarse_PAQ
4,0.004,5.0,-0.326624,-0.812571,-1.025780,-0.419058,0.014856,-0.051715,myo_hex_8_coarse_PAQ
...,...,...,...,...,...,...,...,...,...
79995,0.200,19996.0,4.458240,12.305200,27.821900,6.830940,2.449660,4.413410,myo_hex_coarse_8_PAQ
79996,0.200,19997.0,2.611210,13.955200,28.925900,5.530600,2.116080,5.922020,myo_hex_coarse_8_PAQ
79997,0.200,19998.0,1.488700,15.415700,28.906600,4.097190,1.292730,5.508840,myo_hex_coarse_8_PAQ
79998,0.200,19999.0,0.922051,16.688700,28.844900,2.301980,0.525949,5.989470,myo_hex_coarse_8_PAQ


In [18]:
new_elem_df["vm"]

KeyError: 'vm'

# Transform dataset

- 1: Reduce high values around apex
- 2: Reduce values of penta mesh
- 3: Flatten spikes

In [ ]:
# Get mean values


# Compute aditional info
- Effective stress (von misses)
- Centroidal displacement
- Total displacement

# Compute result
- Mean values
- Mean of max values (threshold) -> bins of 1% get max
- Std

# Compare each result
- Compare each mesh density (coarse, medium, fine) for each result